In [1]:
import os
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
import cv2 as cv


In [2]:
PATH = dict()
PATH['main'] = 'C:\\Users\\javie\\Python\\TFOD\\siamese_face_recognition'
PATH['download'] = os.path.join(PATH['main'],'dataset\\raw')
PATH['raw'] = os.path.join(PATH['download'],'lfw')
PATH['train'] = os.path.join(PATH['main'],'dataset\\train')
PATH['test'] = os.path.join(PATH['main'],'dataset\\test')

In [3]:
directories = os.listdir(PATH['raw'])
files = list()
n_files = list()
for dir in directories:
    file = os.listdir(os.path.join(PATH['raw'],dir))
    n_files.append(len(file))
    for f in file:
        files.append(os.path.join(PATH['raw'], dir, f))

In [4]:
files


['C:\\Users\\javie\\Python\\TFOD\\siamese_face_recognition\\dataset\\raw\\lfw\\Aaron_Eckhart\\Aaron_Eckhart_0001.jpg',
 'C:\\Users\\javie\\Python\\TFOD\\siamese_face_recognition\\dataset\\raw\\lfw\\Aaron_Guiel\\Aaron_Guiel_0001.jpg',
 'C:\\Users\\javie\\Python\\TFOD\\siamese_face_recognition\\dataset\\raw\\lfw\\Aaron_Patterson\\Aaron_Patterson_0001.jpg',
 'C:\\Users\\javie\\Python\\TFOD\\siamese_face_recognition\\dataset\\raw\\lfw\\Aaron_Peirsol\\Aaron_Peirsol_0001.jpg',
 'C:\\Users\\javie\\Python\\TFOD\\siamese_face_recognition\\dataset\\raw\\lfw\\Aaron_Peirsol\\Aaron_Peirsol_0002.jpg',
 'C:\\Users\\javie\\Python\\TFOD\\siamese_face_recognition\\dataset\\raw\\lfw\\Aaron_Peirsol\\Aaron_Peirsol_0003.jpg',
 'C:\\Users\\javie\\Python\\TFOD\\siamese_face_recognition\\dataset\\raw\\lfw\\Aaron_Peirsol\\Aaron_Peirsol_0004.jpg',
 'C:\\Users\\javie\\Python\\TFOD\\siamese_face_recognition\\dataset\\raw\\lfw\\Aaron_Pena\\Aaron_Pena_0001.jpg',
 'C:\\Users\\javie\\Python\\TFOD\\siamese_face_recogni

In [5]:
weights = os.path.join(PATH['main'], "face_detection_yunet_2023mar.onnx")
face_detector = cv.FaceDetectorYN_create(weights, "", (250,250), score_threshold=0.7)

In [6]:
#Detector notation
# 0-1: x, y of bbox top left corner
# 2-3: width, height of bbox
# 4-5: x, y of right eye (blue point in the example image)
# 6-7: x, y of left eye (red point in the example image)
# 8-9: x, y of nose tip (green point in the example image)
# 10-11: x, y of right corner of mouth (pink point in the example image)
# 12-13: x, y of left corner of mouth (yellow point in the example image)
# 14: face score
img1 = cv.imread(files[0])
fr = face_detector.detect(img1)
x, y, width, height = fr[1][0].astype(np.int16)[:4]
fScore = face_detector.detect(cv.imread(files[0]))[1][0][-1]
print(fr)

(1, array([[ 74.8587    ,  62.820034  ,  96.57723   , 121.30079   ,
        101.08038   , 112.12159   , 146.74995   , 112.427246  ,
        126.92289   , 138.44766   , 107.212166  , 155.62553   ,
        144.13467   , 154.11926   ,   0.92166483]], dtype=float32))


In [14]:
def face_rc(img):
    stop = False
    n_faces = 0
    fr = face_detector.detect(img)
    if fr[0] == 0:
        print('There are no faces here!!!')
    else:
        if fr[1].shape[0] == 1:
            x, y, width, height = face_detector.detect(img)[1][0].astype(np.int16)[:4]
            fScore = face_detector.detect(cv.imread(files[0]))[1][0][-1]
            n_faces = 1
        else:
            better_score = 0
            #max_area = 0
            min_dist = 100
            mid = np.array([125.0,125.0])
            for g, guess in enumerate(fr[1]):
                ar = guess[:4]
                cm = ar[:2] + ar[2:]/2.0

                dis = np.sqrt(np.square(cm - mid).sum())
                if min_dist > dis:
                    better_score = g
                    min_dist = dis
                else:
                    pass
                # if max_area < width * height:
                #     better_score = g
                #     max_area = width * height
                # else:
                #     pass

                # if fr[1][better_score][-1] < guess[-1]:
                #     better_score = g
                # else:
                #     pass
            stop = True
            n_faces = fr[1].shape[0]
            x, y, width, height = face_detector.detect(img)[1][better_score].astype(np.int16)[:4]
            fScore = face_detector.detect(img)[1][better_score][-1]
            print('The best choice:', better_score)
        img_l = int(max(105, width, height))
        x_cm = int(x + width/2.0)
        y_cm = int(y + height/2.0)
        x_1 = int(x_cm - img_l/2.0)
        x_2 = int(x_1 + img_l)
        y_1 = int(y_cm - img_l/2.0)
        y_2 = int(y_1 + img_l)
        if min(x_1,y_1) < 0 or max(x_2,y_2)>250:
            if x_1 < 0:
                x_2 = x_2 - x_1
                x_1 = 0
            if x_2 > 250:
                x_1 = x_1 - x_2 + 250
                x_2 = 250
            if y_1 < 0:
                y_2 = x_2 - y_1
                y_1 = 0
            if y_2 > 250:
                y_1 = y_1 - y_2 + 250
                y_2 = 250

        img = cv.rectangle(img, (x_1,y_1), (x_2,y_2), (255,0,0))
        img = cv.putText(img,"Score:{:1.3f}".format(fScore), (10,250), cv.FONT_HERSHEY_SIMPLEX, 0.3, (255,255,255))
        # img = img[y_1:y_2,x_1:x_2]
        # img = cv.resize(img,dsize=(105,105))
        return img, stop, n_faces

In [16]:
for file in files:
    # print(file)
    img = cv.imread(file)
    img, stop, n_faces = face_rc(img)
    #print(n_faces)
    cv.imshow('img',img)
    k = cv.waitKey(200)
    if stop:
        print('Aquí va una!!')
        cv.waitKey(0)
    else:
        if k == ord('q'):
            cv.destroyAllWindows()
            break
        else:
            pass

The best choice: 1
Aquí va una!!
The best choice: 0
Aquí va una!!
The best choice: 0
Aquí va una!!
The best choice: 0
Aquí va una!!
The best choice: 0
Aquí va una!!
The best choice: 0
Aquí va una!!


In [9]:
cv.imshow('img',img1)
cv.waitKey(0)
cv.destroyAllWindows()